In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date, timedelta
import pickle
import requests
import numpy as np
from glicko2 import Glicko2
from trueskill import TrueSkill
from scipy.stats import norm
from collections import defaultdict
%matplotlib inline

In [2]:
df = pd.read_csv('data.csv')

## First look at the data

This is real Counter-Strike match data.

`perid` is the Bayes Esports identifier

- take a look at the available data points
- check data integrity
- how would you restructure it?

## Elo

In [5]:
# write two functions: one that calculates the expected win probability and one that updates the rating
# remember you also need an initial rating value

def expected() -> float:
    pass

def update() -> float:
    pass

In [6]:
# now let's apply that function to the data!

In [7]:
# plotting time: let's look at the team with the most ratings in our dataset

## Testing Elo quality

- We don't need a test set
- Ignore calibration time

## Glicko-2

In [ ]:
tau = .2
sigma = .06
mu = 1500
phi = 350
glicko = Glicko2(mu=mu, phi=phi, tau=tau, sigma=sigma)

In [ ]:
for d, group in df.groupby('date'):
    # each team gets an updated rating for the day, but this rating cannot be used
    # for rating other teams in the same day, hence we keep it as provisional for now
    # it would probably be more interesting to do this on a weekly basis, but then we cannot compare to elo and ts
    provisional_ratings = {}
    # now go through our list of teams and get their series = all matches they had this week
    for team in all_teams:
        relevant = group[(group.winner==team) | (group.loser==team)]
        old_rating = glicko_ratings[team][-1][-1]
        # rewrite the matches in the format taken by the algorithm, which is [(result, opponent_rating)]
        series = []
        for idx, row in relevant.iterrows():
            if row.winner == team:
                series.append((1, glicko_ratings[row.loser][-1][-1]))
            else:
                series.append((0, glicko_ratings[row.winner][-1][-1]))
        new_rating = glicko.rate(old_rating, series)
        provisional_ratings[team] = new_rating
    for k, v in provisional_ratings.items():
        glicko_ratings[k].append(((d, v)))

In [ ]:
def predict_glicko_winner(r1, r2):
    r1_g2 = glicko.scale_down(r1)
    r2_g2 = glicko.scale_down(r2)
    # equivalent to calling Glicko2.reduce_impact
    g = 1 / np.sqrt(1+3*r1_g2.phi **2 / np.pi**2)
    # equivalent to calling Glicko2.expect_score
    E = 1 / (1 + np.exp(-g * (r1_g2.mu - r2_g2.mu)))
    return E
    
wp_glicko = [predict_glicko_winner(i, j) for i, j in zip(df.winner_glicko, df.loser_glicko)]

## Trueskill

In [ ]:
ts = TrueSkill(backend='scipy')